In [63]:
import requests
import joblib
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from pandas import option_context
from tqdm import tqdm
from statsmodels.stats.inter_rater import aggregate_raters
from sklearn.metrics import cohen_kappa_score
from scipy.stats import mannwhitneyu,shapiro,ttest_ind

import cliffsDelta as cd
import GenerateActivities as gat
import important_features as imf

In [ ]:
QUERY_ROOT = "https://api.github.com"
TOKEN = '' # write your GitHub API key here
HEADERS={'Authorization':'token '+TOKEN}

## Section 3

### Question 1

#### Write an automated Python script to identify all GitHub bot actors in the list of contributors, by querying the GitHub REST API users endpoint and extracting the required information to make this decision. 

#### More information: You can search for the "type" key in the obtianed JSON result, If it is "Bot", then it is a bot actor, if it is "User", then it is an account.

In [166]:
# 1. Read the given <sample>.csv file into a pandas DataFrame (df).
# 2. Make sure there are 2 columns, one column name is "contributor" that has the name of the contributor, 
#    and another column name is "bothunter_type" that you will use later on in this assignment.
# 3. Display the DataFrame.

# YOUR CODE BELOW

df = pd.read_csv('sample5.csv')

print(df)

                             contributor bothunter_type
0                           AnjaliSandip          Human
1                        DilumAluthgeBot          Human
2                                 DrDrij          Human
3                       JuliaRegistrator            Bot
4                          LucaMarconato          Human
5                          Robinlovelace          Human
6                          VicidominiLab          Human
7                         alasdairwilson          Human
8                              aulemahal          Human
9                         bioc-issue-bot            Bot
10                         bob-carpenter          Human
11                      codspeed-hq[bot]            Bot
12                          consideRatio          Human
13                       dependabot[bot]            Bot
14                               djs0109          Human
15                              fbourgey          Human
16  galaxyproject-sentryintegration[bot]        

In [167]:
# 1. Store the names of the contributors in a list
# 2. print the list

# YOUR CODE BELOW

contributor = df[["contributor"]]
contributorList = df.contributor.to_list()

print(contributorList)

['AnjaliSandip', 'DilumAluthgeBot', 'DrDrij', 'JuliaRegistrator', 'LucaMarconato', 'Robinlovelace', 'VicidominiLab', 'alasdairwilson', 'aulemahal', 'bioc-issue-bot', 'bob-carpenter', 'codspeed-hq[bot]', 'consideRatio', 'dependabot[bot]', 'djs0109', 'fbourgey', 'galaxyproject-sentryintegration[bot]', 'github-merge-queue[bot]', 'hackmd-hub[bot]', 'hameerabbasi', 'ivirshup', 'jmaspons', 'kalmarek', 'larsoner', 'marinegor', 'michaelbratsch', 'mzkhalid039', 'orbeckst', 'plaplant', 'renovate[bot]', 'review-notebook-app[bot]', 'ropensci-review-bot', 'rouson', 'scverse-bot', 'simeonschaub', 'spyder-bot', 'sympy-bot', 'thyripian', 'weblate', 'willgearty']


In [168]:
# 1. Using a for loop, iterate over each contributor in the list and query the GitHub Users API.
#    You can use "query = f'{QUERY_ROOT}/users/{contributor}'", where QUERY_ROOT is defined at the beginning of this notebook 
#    and 'contributor' is each individual contributor from the list
# 2. Get the response using 'response = requests.get(query, headers=HEADERS)'.
# 3. convert the response to JSON using 'json_response = response.json()'.
# 4. Iterate over each JSON response and get the value of the 'type' key. If it is "Bot" then the contributor is a bot actor, 
#    if "User" then the contributor is an account. You should boolean values to indicate if the contributor is a bot actor (True) or User/Organisation (False)
# 5. Save these results in list of dictionary of the form [{'contributor': <contributor name>, 'app': <boolean value>}, {...}, {...}, {...}].
#    Lets call this list as "app_dict"
# 6. Finally convert this list of dictionary to DataFrame by writing 'pd.DataFrame.from_dict(app_dict)'
# 7. Display the DataFrame. This should have two columns - contributor and app

# YOUR CODE BELOW

app_dict = {}
tab = []
for c in range(len(contributorList)):
    query = f'{QUERY_ROOT}/users/{contributorList[c]}'
    response = requests.get(query, headers=HEADERS)
    json_response = response.json()
    if(json_response['type'] == "User" or json_response['type'] == "Organisation"):
        tab.append(False)
    else:
        tab.append(True)
    
app_dict = [{'contributor': contributor, 'app': status} for contributor, status in zip(contributorList, tab)]    
app_data = pd.DataFrame.from_dict(app_dict)
display(app_data)


,contributor,app
0,AnjaliSandip,False
1,DilumAluthgeBot,False
2,DrDrij,False
3,JuliaRegistrator,False
4,LucaMarconato,False
5,Robinlovelace,False
6,VicidominiLab,False
7,alasdairwilson,False
8,aulemahal,False
9,bioc-issue-bot,False


### Question 2

#### Add a new 'actor' column in the CSV file to store this information. You can write True if it is a bot actor and False if it is not.

#### Report on the total number of GitHub bot actors, and User accounts present in the list of accounts of your dataset.

In [171]:
# Merge the app DataFrame to df by writing 'pd.merge(df, <app df>, on='contributor'). This is similar to SQL join on primary key 'contributor'.
# The resultant df should have 3 columns - contributor, bothunter_type and user_type.

# YOUR CODE BELOW

mergeDF = pd.merge(df, app_data, on='contributor')
display(mergeDF)


,contributor,bothunter_type,app
0,AnjaliSandip,Human,False
1,DilumAluthgeBot,Human,False
2,DrDrij,Human,False
3,JuliaRegistrator,Bot,False
4,LucaMarconato,Human,False
5,Robinlovelace,Human,False
6,VicidominiLab,Human,False
7,alasdairwilson,Human,False
8,aulemahal,Human,False
9,bioc-issue-bot,Bot,False


### Question 3

#### For each bot actor, write their purpose, the task that they automate by looking at their GitHub profile, homepage, information on the GitHub Marketplace, or other documentation that you can find online.


## Section 4

### Question 1

#### Write an automated script to use BIMBAS (Bot Identification Model Based on Activity Sequences) to obtain a prediction of the GitHub contributor type (Bot or Human) for the contributors present in your dataset.

#### Folow the steps given in each cell to use BIMBAS and obtain predictions.

### Read all the events and select the events performed by the contributors present in the given set

In [188]:
# 1. Read the csv file of events (all_events.csv) provided along with this notebook
# 2. Convert created_at column to datetime format 
#    One possible way is to use lambda function: "events.assign(created_at=lambda d: pd.to_datetime(d.created_at, unit='ms'))"
# 3. Get the contributors provided to you in a list or use the contributors list that you created in Section 3 Question 1. 
#    e.g., ['contributor1', 'contributor2', 'contributor3',...]
# 4. Select the events performed by these contributors alone. Do not consider all the events. You should consider only the events
#    performed by the contributors provided to you. Hint: you can use the df.query function
# 5. Display the considered events.

# YOUR CODE BELOW

df2 = pd.read_csv('all_events.csv')
tab2 = []
df2 =df2.assign(created_at= lambda d: pd.to_datetime(d.created_at, unit='ms'))
for e in range(len(df2)):
    if(df2.iloc[e].login in contributorList):
        tab2.append(df2.iloc[e])

tab2 = pd.DataFrame(tab2)
display(tab2)

,Unnamed: 0,event_id,event_type,login,repository,created_at,action,PR_number,state,PR_node_id,...,comment_node_id,merged,ref,ref_type,issue_number,issue_node_id,issue_closed_at,tag_name,release_node_id,org
163,163,39747766396,CreateEvent,dependabot[bot],SciML/EasyModelAnalysis.jl,2024-07-01 00:34:53,NaN,NaN,NaN,NaN,...,NaN,NaN,dependabot/github_actions/actions/add-to-proje...,branch,NaN,NaN,NaN,NaN,NaN,SciML
164,164,39747766717,PullRequestEvent,dependabot[bot],SciML/EasyModelAnalysis.jl,2024-07-01 00:34:55,opened,261.0,open,PR_kwDOI1YYHs50AC5Y,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SciML
369,369,39749032522,CreateEvent,renovate[bot],OSGeo/grass,2024-07-01 02:04:56,NaN,NaN,NaN,NaN,...,NaN,NaN,renovate/pdal-pdal-2.x,branch,NaN,NaN,NaN,NaN,NaN,OSGeo
370,370,39749033175,PullRequestEvent,renovate[bot],OSGeo/grass,2024-07-01 02:04:58,opened,3960.0,open,PR_kwDOCykpn850AZPk,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OSGeo
398,398,39749355018,CreateEvent,dependabot[bot],pymc-devs/pymc,2024-07-01 02:26:02,NaN,NaN,NaN,NaN,...,NaN,NaN,dependabot/github_actions/docker/build-push-ac...,branch,NaN,NaN,NaN,NaN,NaN,pymc-devs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358282,358282,42414205596,CreateEvent,dependabot[bot],scientific-python/upload-nightly-action,2024-09-30 22:21:10,NaN,NaN,NaN,NaN,...,NaN,NaN,dependabot/github_actions/actions-aa3137d618,branch,NaN,NaN,NaN,NaN,NaN,scientific-python
358283,358283,42414206104,PullRequestEvent,dependabot[bot],scientific-python/upload-nightly-action,2024-09-30 22:21:11,opened,103.0,open,PR_kwDOJmNtS859LUGn,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,scientific-python
358335,358335,42414900284,IssueCommentEvent,bioc-issue-bot,Bioconductor/Contributions,2024-09-30 23:00:13,created,NaN,open,NaN,...,IC_kwDOAyn0c86OHdfO,NaN,NaN,NaN,3542.0,I_kwDOAyn0c86WLedB,NaN,NaN,NaN,Bioconductor
358385,358385,42415261485,CommitCommentEvent,JuliaRegistrator,SciML/LineSearch.jl,2024-09-30 23:21:33,NaN,NaN,NaN,NaN,...,CC_kwDOLCAB0M4IyT9_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SciML


### Execute BIMBAS to obtain the predictions

In [186]:
# Do not modify this cell
# Use the following function the code below to obtain your predictions

def execute_bimbas(contributors_list, selected_events):
    '''
    args: contributors_list (list) - list of contributors in the dataset provided to you
          selected_events (DataFrame) - DataFrame of events performed by the considered contributors
    return: bimbas_prediction (DataFrame) - DataFrame of preditions along with the confidence in prediction for each contributor
    '''
    result=pd.DataFrame()
    temp_list = []
    bimbas = joblib.load('bimbas.joblib')
    date_limit = pd.to_datetime(selected_events.created_at.max()) + pd.DateOffset(-90)
    
    for contributor in tqdm(contributors_list):
        # c = contributors[contributor]
        activities = gat.activity_identification(selected_events.query('login==@contributor and created_at>=@date_limit'))
        activity_features = (
                            imf.extract_features(activities)
                            .set_index([[contributor]])
                            )
        # features = pd.concat([features,activity_features])
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            probability = bimbas.predict_proba(activity_features)[0][1]
        if(probability <= 0.5):
            contributor_type = 'Human'
        else:
            contributor_type = 'Bot'
        confidence = (abs(probability - 0.5)*2).round(3)
    
        pred = activity_features.set_index([[contributor]]).assign(
            prediction=contributor_type,
            confidence = confidence,
            )
        temp_list.extend([{'contributor':contributor,'bimbas_type':contributor_type,'confidence':confidence}])
    
    bimbas_prediction = pd.DataFrame.from_dict(temp_list)

    return(bimbas_prediction)

In [190]:
# Write your code here to call execute_bimbas function. 
# Pass the contributors list and the filtered events DataFrame as argument to "execute_bimbas" function
# Prediction returned by BIMBAS will have "contributor", "bimbas_type" and "confidence"
# Note: Print the prediction provided by BIMBAS - else this cell will not be graded.

# YOUR CODE BELOW

BB = execute_bimbas(contributorList, tab2)
display(BB)

  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:18<00:00,  2.19it/s]


,contributor,bimbas_type,confidence
0,AnjaliSandip,Bot,0.927
1,DilumAluthgeBot,Bot,0.894
2,DrDrij,Human,0.836
3,JuliaRegistrator,Bot,0.886
4,LucaMarconato,Human,0.520
5,Robinlovelace,Human,0.921
6,VicidominiLab,Bot,0.861
7,alasdairwilson,Human,0.911
8,aulemahal,Human,0.943
9,bioc-issue-bot,Bot,0.548


### Create a column in your predictions DataFrame to have predictions provided by BIMBAS. 
### Now your DataFrame should have the following columns - contributor, bothunter_type, app, bimbas_type, confidence 

### Merging bothunter and bimbas predictions

In [191]:
# Merge the predictions provided by BIMBAS to the resultant dataset of Section 3. This final DataFrame should have the following columns - 
# contributor, bothunter_type, user_type, bimbas_type, and confidence.

mergeDF2 = pd.merge(mergeDF, BB, on='contributor')
display(mergeDF2)

,contributor,bothunter_type,app,bimbas_type,confidence
0,AnjaliSandip,Human,False,Bot,0.927
1,DilumAluthgeBot,Human,False,Bot,0.894
2,DrDrij,Human,False,Human,0.836
3,JuliaRegistrator,Bot,False,Bot,0.886
4,LucaMarconato,Human,False,Human,0.520
5,Robinlovelace,Human,False,Human,0.921
6,VicidominiLab,Human,False,Bot,0.861
7,alasdairwilson,Human,False,Human,0.911
8,aulemahal,Human,False,Human,0.943
9,bioc-issue-bot,Bot,False,Bot,0.548


### Question 2
#### Using Cohen's Kappa compute and report the interrater agreement score between the labels computed by both the bot identification approaches. Mention your interpretation of Cohen's Kappa

In [203]:
# Hint: use the cohen_kappa_score library
# YOUR CODE BELOW

CK = cohen_kappa_score(mergeDF2.bothunter_type.to_list(), mergeDF2.bimbas_type.to_list())
display(CK)
print("With a score of 6.5, the Ck tell us there are a substantial agreement between bot identification approches")

np.float64(0.65)

With a score of 6.5, the Ck tell us there are a substantial agreement between bot identification approches


### Question 3
#### Determine the final type of each contributor. Whenever user_type column has the value "User" check if both bimbas_type and bothunter_type give the same prediction, then consider it as your final prediction in "acc_type" column.
#### For the contributors that have different predictions, i.e. bimbas_type is not same as bothunter_type, then make a manual verification and consider that as the type for that contributor. Add an extra column to the DataFrame ﬁle called 'manual' that has the label determined by you. 
#### For manual verification you can make use the GitHub UI and the GitHub API https://api.github.com/users/username

#### Whenever user_type column has the value "Bot", directly write your final acc_type as "Bot Actor"

#### More information:
For each contributor, you can look at their activities in GitHub UI, look at their latest events using GitHub Events API - https://api.github.com/users/<contributor>/events, to make a decision on their type.

In addition, you can also write a very small reason why do you think they are bot or human (e.g., same activity at regular interval so Bot, files committed and code modified looks like Human, comments look like Human, comment look like Bot, test report looks automated so Bot, and so on....). This will be useful to answer your next question.

In [43]:
# Write the code for your manual decision here
# example : 
# manual_list_dict = [{'contributor': '<contributor name>', 'manual': '<your prediction>'}, {....}, {....}, {....}]
# pd.DataFrame.from_dict(manual_list_dict)

# YOUR CODE BELOW


In [44]:
# Write your code below here to determine the final type of contributor.
# add column named 'acc_type' and write your final decision in it. Final decision is majority of three types - bimbas_type, bothunter_type, manual.


### Question 4
#### Study and report the purpose of these identified bots and humans in the repository assigned to you.
#### For example, the purpose can be based on but not limited to - 
##### 1) the type of activities that they are performing (releasing a version on every Sunday, updating the documentation), 
##### 2) type of comments they are posting (reviewing code, test summary report), 
##### 3) when are they triggered (when a new PR is created, when someone in the project ask the bot to merge the code) and so on.  
#### Note: 
##### The purpose can be other than what is provided in examples above.

## Section 5

### Question 1
#### Use the filtered events file that has the events performed by the contributors provided to you
#### Group each event into the following four categories

Issues: IssueCommentEvent, IssuesEvent  
Pull Requests: PullRequestEvent, PullRequestReviewCommentEvent  
Commits: CommitCommentEvent, PushEvent  
Repository: CreateEvent, DeleteEvent, ForkEvent, GollumEvent, MemberEvent, PublicEvent, ReleaseEvent, SponsorshipEvent, WatchEvent  

#### Hint:
1. Add a column called event_group that mentions which event group does that event belong to. Each event (row) should correspond to an event group.  
2. Then perform groupby on ['login','event_group'],  
3. use .agg (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html) to count the number of events performed by each contributor in each group,  
4. use pivot with the required arguments (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html). An example is given [here](#pivot_example) in question 2(b), and
5. Reset index and rename axis with None
7. Finally merge it with your 'acc_type' field from the DataFrame you created in Section4, drop contributor, and fillna with 0
8. The final DataFrame should have the following columns - login, PR, commit, issue, repo, acc_type

The final DataFrame should be looking like the following  
![](event_group.png)

In [45]:
# In the resultant DataFrame - each row should corespond to a contributor, and the columns should have all the event groups  
# and the type of the contributor (that you decided in the previous DataFrame) and the values should be the number 
# of events of that event event group the contriubutor has performed.

issue_group = ['IssueCommentEvent', 'IssuesEvent']
pr_group = ['PullRequestEvent', 'PullRequestReviewCommentEvent']
commit_group = ['CommitCommentEvent', 'PushEvent']
repo_group = ['CreateEvent', 'DeleteEvent', 'ForkEvent', 'GollumEvent', 'MemberEvent', 'PublicEvent', 'ReleaseEvent', 'SponsorshipEvent', 'WatchEvent']

In [46]:
# YOUR CODE BELOW


In [47]:
# YOUR CODE BELOW


### Question 2 (a)

#### Compute the median number of events per event group for Bot+Bot actors and Humans and write in DataFrame.

Row should correspond to type (Bot_BotActor and Human), Column should have Event group name and the values should be the median value of Bot_BotActor or Human for that particular event group. An example is given below

In [48]:
# For example:
medians = [{'event_group': 'event_group1', 'median': 'val1', 'acc_type': 'Bot_app'}, 
           {'event_group': 'event_group1', 'median': 'val2', 'acc_type': 'Human'},
           {'event_group': 'event_group2', 'median': 'val3', 'acc_type': 'Bot_app'},
           {'event_group': 'event_group2', 'median': 'val4', 'acc_type': 'Human'},
           {'event_group': 'event_group3', 'median': 'val5', 'acc_type': 'Bot_app'},
           {'event_group': 'event_group3', 'median': 'val6', 'acc_type': 'Human'},
           {'event_group': 'event_group4', 'median': 'val7', 'acc_type': 'Bot_app'},
           {'event_group': 'event_group5', 'median': 'val8', 'acc_type': 'Human'}]
df_medians = pd.DataFrame.from_dict(medians)
df_medians

,event_group,median,acc_type
0,event_group1,val1,Bot_app
1,event_group1,val2,Human
2,event_group2,val3,Bot_app
3,event_group2,val4,Human
4,event_group3,val5,Bot_app
5,event_group3,val6,Human
6,event_group4,val7,Bot_app
7,event_group5,val8,Human


In [49]:
# YOUR CODE BELOW

### Question 2 (b)

Plot a heatmap of the DataFrame using seaborn - 
1. First convert the dataframe to the required format using pivot, example is given below
2. plot using seaborn - sns.heatmap(df_medians, annot=True, vmin=0, vmax=300, cmap="crest"). More details: https://seaborn.pydata.org/generated/seaborn.heatmap.html)

#### pd.pivot example:
<a id='pivot_example'></a>

In [50]:
# Main DataFrame
medians = [{'event_group': 'event_group1', 'median': 'val1', 'acc_type': 'Bot'}, 
           {'event_group': 'event_group1', 'median': 'val2', 'acc_type': 'Human'},
           {'event_group': 'event_group2', 'median': 'val3', 'acc_type': 'Bot'},
           {'event_group': 'event_group2', 'median': 'val4', 'acc_type': 'Human'},
           {'event_group': 'event_group3', 'median': 'val5', 'acc_type': 'Bot'},
           {'event_group': 'event_group3', 'median': 'val6', 'acc_type': 'Human'},
           {'event_group': 'event_group4', 'median': 'val7', 'acc_type': 'Bot'},
           {'event_group': 'event_group5', 'median': 'val8', 'acc_type': 'Human'}]
df_medians = pd.DataFrame.from_dict(medians)
df_medians

,event_group,median,acc_type
0,event_group1,val1,Bot
1,event_group1,val2,Human
2,event_group2,val3,Bot
3,event_group2,val4,Human
4,event_group3,val5,Bot
5,event_group3,val6,Human
6,event_group4,val7,Bot
7,event_group5,val8,Human


In [51]:
# pivot the main DataFrame
df_medians.pivot(index='acc_type', columns='event_group', values='median')#.reset_index().rename_axis(None,axis=1)

event_group,event_group1,event_group2,event_group3,event_group4,event_group5
acc_type,,,,,
Bot,val1,val3,val5,val7,NaN
Human,val2,val4,val6,NaN,val8


In [52]:
# YOUR CODE BELOW

In [53]:
# YOUR CODE BELOW


#### What is the difference that you observe between Bots+Bot actors and Humans?

#### What is the differnce that you observe between Event groups?

#### What is the difference that you observe between Bots+Bot actors and Humans and Event groups all considering at the same time?

### Question 3

#### Create boxen plots to visualise the distribution of number of events in each event group. 
#### For more information you can visit - https://seaborn.pydata.org/generated/seaborn.boxenplot.html#seaborn.boxenplot 
#### You should highlight the data points that correspond to bots using a stripplot in seaborn. https://seaborn.pydata.org/tutorial/categorical.html#categorical-tutorial  
#### Interpret the results of the visualisation.

In [54]:
# YOUR CODE BELOW - Visualize number of events from pull request event group


In [55]:
# YOUR CODE BELOW - Visualise number of events from issue event group


In [56]:
# YOUR CODE BELOW - Visualise number of events from commit event group


In [57]:
# YOUR CODE BELOW - Visualise number of events from repo event group


### Question 4.1

#### Statistical identify whether number of events in each event group is normally distributed or not.
#### Null hypothesis - $H_0$: Sample comes from the data that has normal distribution.
#### Use Shapiro-Wilk test for this purpose. Use the p-value with a threshold of 0.05 to determine whether $H_0$ can be rejected with statistical significance or not. 

#### Use shapiro from scipy.stats to perform this test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html)

In [58]:
# YOUR CODE BELOW

### Question 4.2
#### Determine if there is any statistical diﬀerence in number of events between the identified event groups. Perform this test on all pairs of event groups. 
#### Null hypothesis - $H_0$: Any two event group come from the same population.
#### If $H_0$ is rejected in Shapiro-Wilk test (at least for one of the two event groups considered for test), use the Mann-Whitney U statistical test for this purpose. If $H_0$ is not rejected in Shapiro-Wilk test, use the independent t-test for this purpose.  
#### In any case, use the p-value with a threshold of 0.01 to determine whether $H_0$ can be rejected with statistical significance. 

#### Use mannwhitneyu from scipy.stats to perform Mann-Whitney U test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html) 
#### or 
#### ttest_ind from scipy.stats to perform independent t-test https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html.

#### More information:

You can pass the following arguments to mannwhitneyu from scipy.stats - (method='exact', nan_policy='omit'). For ttest_ind you can use - (nan_policy='omit')

In [59]:
# YOUR CODE BELOW

### Question 4.3
#### Each time you reject the null hypothesis $H_0$, quantify the effect size of the diﬀerence between the groups using cliﬀ’s delta ($\delta$). 
#### To calculate cliﬀ’s delta, you can pass the list of values to cliﬀ delta.py file given in the repository. E.g., cliffsDelta.cliffsDelta(list of values, list of values). This will return the effect size.
#### Refer to the table given in the TP document and mention your interpretation (negligible, small, medium, large).

In [60]:
# YOU CODE BELOW